In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

2025-04-20 00:05:15.401810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


Logistic Regression
-

In [16]:
data = [[2, 0], [4, 0], [6, 0], [8, 1], [10, 1], [12, 1], [14, 1]]
x_data = [x_row[0] for x_row in data]
y_data = [y_row[1] for y_row in data]

a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

In [18]:
#sigmoid
y = 1/(1+np.e**-(a*x_data + b))

#loss
loss = -tf.reduce_mean(np.array(y_data) * tf.log(y) + (1-np.array(y_data))* tf.log(1-y))

learning_rate = 0.5
gradient_descent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(60001):
        sess.run(gradient_descent)
        if i  % 6000 == 0:
            print("Epoch: %.f, loss = %.4f, slope a = %.4f, y-intercept = %.4f" % (i, sess.run(loss), sess.run(a), sess.run(b)))

    calc_a = sess.run(a)
    calc_b = sess.run(b)

def NewDatacalc(new_x_data):
    return 1 / (1 + np.e**-(calc_a * new_x_data + calc_b))

print(NewDatacalc(5))
print(NewDatacalc(7))
print(NewDatacalc(13))

/var/folders/x5/47g9ybwn3dj2qvmd4l8th0x80000gn/T/ipykernel_8322/2318995901.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Epoch: %.f, loss = %.4f, slope a = %.4f, y-intercept = %.4f" % (i, sess.run(loss), sess.run(a), sess.run(b)))


Epoch: 0, loss = 4.0817, slope a = 2.4706, y-intercept = -0.3620
Epoch: 6000, loss = 0.0152, slope a = 2.9230, y-intercept = -20.3114
Epoch: 12000, loss = 0.0081, slope a = 3.5648, y-intercept = -24.8081
Epoch: 18000, loss = 0.0055, slope a = 3.9564, y-intercept = -27.5511
Epoch: 24000, loss = 0.0041, slope a = 4.2385, y-intercept = -29.5268
Epoch: 30000, loss = 0.0033, slope a = 4.4590, y-intercept = -31.0705
Epoch: 36000, loss = 0.0028, slope a = 4.6399, y-intercept = -32.3371
Epoch: 42000, loss = 0.0024, slope a = 4.7933, y-intercept = -33.4107
Epoch: 48000, loss = 0.0021, slope a = 4.9263, y-intercept = -34.3424
Epoch: 54000, loss = 0.0019, slope a = 5.0439, y-intercept = -35.1653
Epoch: 60000, loss = 0.0017, slope a = 5.1491, y-intercept = -35.9020
[3.88235158e-05]
[0.53537986]
[1.]


Multiple Logistic Regression
-

In [20]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

#시드는 실행할 때마다 같은 결과를 출력해줌
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

# x, y의 데이터 값
x_data = np.array([[2, 3], [4, 3],[6, 4],[8, 6], [10, 7], [12, 8], [14, 9]])
y_data = np.array([0, 0, 0, 1, 1, 1, 1]).reshape(7, 1) #[0], [0], [0], [1], [1], [1], [1]

#입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 2]) #x 두개 짜리 데이터 그릇
Y = tf.placeholder(tf.float64, shape=[None, 1]) #y 한개 짜리 데이터 그릇

#a 안에 a1 a2 값 모두 저장
a = tf.Variable(tf.random_normal([2, 1], dtype=tf.float64))
b = tf.Variable(tf.random_normal([1], dtype=tf.float64))

#y 시그모이드 함수 방정식 사용
y = tf.sigmoid(tf.matmul(X, a)+ b)

#오차 함수
loss = -tf.reduce_mean(Y*tf.log(y) + (1 - Y) * tf.log(1-y))

#학습률 값
learning_rate = 0.1

#오차 최소화 값
gradient_descent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

predicted = tf.cast(y>0.5, dtype=tf.float64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(3001):        
        a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict = {X: x_data, Y: y_data})
        if (i+1) % 300 == 0:
            print(f'step={i+1}, a1={a_[0]}, a2={a_[1]}, b={b_}, loss={loss_}')
    
    print("predicted=", sess.run(predicted, feed_dict={X:x_data}))
    p_val, h_val = sess.run([predicted, y], feed_dict={X:[[1, 5], [10, 5], [4, 5]]})
    print("check predicted=", p_val)
    print("check hypothesis=", h_val)

    h, c, a = sess.run([y, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\n Hypothesis: ", h, "\nCorrect(Y): ", c, "\nAccuracy: ", a)

    new_x = np.array([7, 6.]).reshape(1, 2)
    new_y, new_y_result = sess.run([y, predicted], feed_dict={X: new_x})

    print(f"공부한 시간: {new_x[:, 0]} , {new_x[:, 1]}")
    print("합격 가능성: %6.2f%%" %(new_y*100))
    print(f"예측 결과:", new_y_result[0])


step=300, a1=[1.04453619], a2=[-0.8111726], b=[-2.78782518], loss=0.24920464762582648
step=600, a1=[0.93470692], a2=[-0.41186888], b=[-4.10121265], loss=0.18541079345399925
step=900, a1=[0.80168377], a2=[-0.03562521], b=[-5.09986046], loss=0.14665520570049115
step=1200, a1=[0.67540874], a2=[0.2960461], b=[-5.90621985], loss=0.1206863657982371
step=1500, a1=[0.56286146], a2=[0.58425401], b=[-6.58233881], loss=0.10220458438379738
step=1800, a1=[0.46453787], a2=[0.8348311], b=[-7.1643961], loss=0.08846481070224202
step=2100, a1=[0.379027], a2=[1.05397569], b=[-7.67537416], loss=0.07789698477150467
step=2400, a1=[0.30449816], a2=[1.24713424], b=[-8.13076902], loss=0.06954206209936582
step=2700, a1=[0.2392001], a2=[1.41881062], b=[-8.54151066], loss=0.0627847181576291
step=3000, a1=[0.1816104], a2=[1.57264621], b=[-8.91559007], loss=0.05721427079587034
predicted= [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
check predicted= [[0.]
 [1.]
 [0.]]
check hypothesis= [[0.29531881]
 [0.68204343]
 [0.

/var/folders/x5/47g9ybwn3dj2qvmd4l8th0x80000gn/T/ipykernel_8322/1150720325.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("합격 가능성: %6.2f%%" %(new_y*100))


In [23]:
#logistic regression

In [25]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Y = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1 ]
y_data = [0.1, 0.2, 0.3, 0.1, 0.2, 0.6, 0.7, 0.8, 0.6, 0.9]

y = np.array(y_data)
predicted = tf.cast(y>0.5, dtype=tf.float64)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))

with tf.Session() as sess:
    temp1 = sess.run(tf.equal(predicted, Y))
    print(temp1)
    temp2 = sess.run(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))
    print(temp2)
    result = sess.run(accuracy)
    print(result)

[ True  True  True  True  True False  True  True  True  True]
[1. 1. 1. 1. 1. 0. 1. 1. 1. 1.]
0.9
